### Import the CNN Util and libraries needed
We have the util to make it easy to create and try new variations of the CNN model and be consistent with how we're analyzing and evaluating it.

In [ ]:
# Import necessary libraries
import cnn_utils
from keras import layers, models, optimizers, callbacks
from keras.src.legacy.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50, EfficientNetB0, VGG16
from keras.applications.imagenet_utils import preprocess_input

### Load the data

In [ ]:
data_dict = cnn_utils.load_cifar10_from_tar()

### Preporcess the data

In [ ]:
data = cnn_utils.preprocess_data(data_dict)

### Let's do a quick visualization of sample images (to also ensure we still have the correct shape)

In [ ]:
cnn_utils.visualize_data_samples(data)


### Data processing for Transfer Learning
- Converts back to [0, 255]: Your data was normalized to [0, 1], but ImageNet pre-trained models expect the original [0, 255] pixel range
- Applies ImageNet preprocessing: Uses preprocess_input() which applies model-specific normalization (e.g., ResNet uses different normalization than VGG)

In [ ]:
def preprocess_for_transfer_learning(data):
    data['X_train'] = data['X_train'] * 255.0
    data['X_val'] = data['X_val'] * 255.0
    data['X_test'] = data['X_test'] * 255.0
    
    # Apply ImageNet preprocessing (model-specific normalization)
    data['X_train'] = preprocess_input(data['X_train'])
    data['X_val'] = preprocess_input(data['X_val'])
    data['X_test'] = preprocess_input(data['X_test'])
    
    return data

# Apply transfer learning preprocessing
data = preprocess_for_transfer_learning(data)
print("Data preprocessed for transfer learning")

### Data Augmentation

Moderate geometric augmentation that applies realistic transformations to training images:
- Rotation (±15°), shifting (10% in each direction), and zooming (±10%) simulate natural camera angle and distance variations
- Horizontal flipping doubles the dataset by creating mirror images (works well for CIFAR-10 since objects like cars/planes look realistic when flipped)


In [ ]:
def create_augmentation():
    return ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        zoom_range=0.1
    )

augmentation = create_augmentation()
augmentation.fit(data['X_train'])

### Let's define our CNN model (architecture)
Deeper, more sophisticated architecture for higher accuracy
Structure:
- 3 convolutional blocks (64→128→256 filters)
- BatchNormalization after each conv layer
- Progressive dropout (0.3→0.4→0.5)
- Large dense layer (512 neurons)

### Create transfer model

In [ ]:
def create_transfer_model(base_model_name='resnet50', num_classes=10):
    """
    Create transfer learning model with frozen base and custom classifier
    """
    # Choose base model
    if base_model_name == 'resnet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
    elif base_model_name == 'efficientnet':
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
    elif base_model_name == 'vgg16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
    
    # Freeze the base model
    base_model.trainable = False
    
    # Build the complete model
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(), 
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model, base_model

# Create the model
model, base_model = create_transfer_model('resnet50')
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

cnn_utils.print_model_summary(model)

### Phase 1 Training (Frozen Base)


In [ ]:
print("=== PHASE 1: Training classifier with frozen base ===")
# Train with frozen base model
history_phase1 = cnn_utils.train_model(
    model, 
    data, 
    augmentation=augmentation,
    epochs=20,
    batch_size=32
)


### Phase 2 Training (Fine-tuning)

In [ ]:
print("=== PHASE 2: Fine-tuning with unfrozen base ===")

# Unfreeze the base model for fine-tuning
base_model.trainable = True

# Recompile with lower learning rate for fine-tuning
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5),  # Much lower learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Continue training with fine-tuning
history_phase2 = cnn_utils.train_model(
    model,
    data,
    augmentation=augmentation,
    epochs=30,
    batch_size=32
)

### Combined History

In [ ]:
# Combine training histories for complete visualization
def combine_histories(hist1, hist2):
    """Combine two training histories"""
    combined = {}
    for key in hist1.history.keys():
        combined[key] = hist1.history[key] + hist2.history[key]
    
    # Create a mock history object
    class CombinedHistory:
        def __init__(self, history_dict):
            self.history = history_dict
    
    return CombinedHistory(combined)

# Combine both training phases
combined_history = combine_histories(history_phase1, history_phase2)

### Let's show the evaluation result

In [ ]:
cnn_utils.evaluate_model(model, data, combined_history)